In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ccdataset/counters.csv


In [36]:
df = pd.read_csv("/kaggle/input/ccdataset/counters.csv")

In [122]:
print(len(df['id'].unique().tolist()))
print(len(df[df['event']=='LAME']['id'].unique().tolist()))


15322
2573


In [53]:
def counter_group(name, labels):
    return [label for label in labels if label.count(name) != 0]

df['ILLMISC'] = [0] * df.shape[0]
group = 'ILLMISC'
for label in counter_group(group, df.columns):
    df[group] = (df[group] + df[label]) != 0
df.drop(columns=['ILLMICS'])

,Unnamed: 0,event_id,id,sex,lactation_number,birth_difficult,days_no_milking,days_pregnant,group_id,prev_group_id,...,ILLMISC_RUBEZ,ILLMICS_DMG,ILLMISC_GASTRO,ILLMISC_PNEVMO,ATR,HEALTH,POT,VAC,VACVIR,ILLMISC
0,0,0,146,1,6,1,62,277,18,18,...,False,False,False,False,False,False,False,False,False,False
1,1,1,146,1,6,1,62,277,18,18,...,False,False,False,False,False,False,False,True,False,False
2,2,2,146,1,6,1,62,277,18,18,...,False,False,False,False,False,False,False,True,False,False
3,3,3,291,1,6,1,68,0,23,23,...,False,False,False,False,False,False,False,False,False,False
4,4,4,291,1,6,1,68,0,23,23,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592302,592302,592302,912269,1,1,1,0,239,5,7,...,False,False,False,False,False,False,False,False,False,False
592303,592303,592303,912269,1,1,1,0,239,5,7,...,False,False,False,False,False,False,False,False,False,False
592304,592304,592304,912269,1,1,1,0,239,7,7,...,False,False,False,False,False,False,False,False,False,False
592305,592305,592305,912270,1,1,1,0,273,17,17,...,False,False,False,False,False,False,False,True,False,False


# Связь заболеваемостью маститом и наличия атрофированных сосков

In [37]:
subjects = df[df['BROKE_ATR'] == 1]

print(subjects[['id', 'date', 'event']].head(100))
ids = subjects['id'].tolist()
idd = subjects['date'].tolist()

for i in ids:
    print(df[(df['id'] == i) & ((df['event'] == 'MAST') | (df['event'] == 'BROKE'))][['date','event', 'remark']])
    print('-------------------')

            id        date  event
402756  706024  2021-07-17  BROKE
402811  811201  2021-07-17  BROKE
403373  811062  2021-07-19  BROKE
405235  703149  2021-07-22  BROKE
405353  904154  2021-07-22  BROKE
405369  909208  2021-07-22  BROKE
428313  706085  2021-09-16  BROKE
              date  event   remark
338059  2021-03-06   MAST    КМ1_3
389893  2021-06-18   MAST   СКМ5_1
391267  2021-06-22   MAST    КМ4_1
396150  2021-06-29   MAST   СКМ4_1
400566  2021-07-11   MAST   КМ4_12
402755  2021-07-17  BROKE  ТУГОДОЙ
402756  2021-07-17  BROKE     ATP4
-------------------
              date  event remark
387629  2021-06-11   MAST  КМ3_4
390457  2021-06-20   MAST  КМ5_4
393259  2021-06-28   MAST  КМ1_4
402811  2021-07-17  BROKE   АТР4
-------------------
              date  event remark
286109  2020-11-12   MAST  КМ4_1
372669  2021-05-17   MAST  ТМ9_2
402893  2021-07-18   MAST     МВ
403373  2021-07-19  BROKE   АТР1
-------------------
              date  event  remark
386595  2021-06-09   MAS

# Влияние послеродовых заболеваний на заболеваемость маститом

In [39]:
illed = df[(df['PARES'] == 1) | (df['KETOS'] == 1) | (df['METR'] == 1) | (df['RP'] == 1)]
illed_id   = illed['id'].tolist()
illed_age = illed['age'].tolist()

sids  = illed['id'].unique().tolist() 
gids  = {sids[i] : i for i in range(len(sids))}

weak_30 = [0]*len(sids)
weak_60 = [0]*len(sids)
weak_90 = [0]*len(sids)
weakall = [0]*len(sids)

dfMAST = df[df['event'] == 'MAST']

for i in range(len(illed_id)):
    idd = illed_id[i]
    age = illed_age[i]
    
    if weak_30[gids[idd]] == 1:
        continue
    
    cond_30 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 30)
    cond_60 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 60)
    cond_90 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 90)
    condall = (dfMAST['id'] == idd) & (dfMAST['age'] > age)
    
    if dfMAST[cond_30].shape[0] > 0:
        weak_30[gids[idd]] = 1  
    if dfMAST[cond_60].shape[0] > 0:
        weak_60[gids[idd]] = 1  
    if dfMAST[cond_90].shape[0] > 0:
        weak_90[gids[idd]] = 1  
    if dfMAST[condall].shape[0] > 0:
        weakall[gids[idd]] = 1  
print(np.count_nonzero(weak_30)/len(sids))
print(np.count_nonzero(weak_60)/len(sids))
print(np.count_nonzero(weak_90)/len(sids))
print(np.count_nonzero(weakall)/len(sids))

print(np.count_nonzero(weak_90)/dfMAST.groupby('id').sum().shape[0])
print(np.count_nonzero(weakall)/dfMAST.groupby('id').sum().shape[0])

0.09345794392523364
0.13978922250944523
0.17876317359315969
0.35911712069994034
0.3249006143838092
0.6526924466931695


In [44]:
illed = df[df['PARES'] == 1]
illed_id   = illed['id'].tolist()
illed_age  = illed['age'].tolist()

sids  = illed['id'].unique().tolist() 
gids  = {sids[i] : i for i in range(len(sids))}

weak_30 = [0]*len(sids)
weak_60 = [0]*len(sids)
weak_90 = [0]*len(sids)
weakall = [0]*len(sids)

for i in range(len(illed_id)):
    idd = illed_id[i]
    age = illed_age[i]
    
    if weak_30[gids[idd]] == 1:
        continue
    
    cond_30 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 30)
    cond_60 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 60)
    cond_90 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 90)
    condall = (dfMAST['id'] == idd) & (dfMAST['age'] > age)
    
    if dfMAST[cond_30].shape[0] > 0:
        weak_30[gids[idd]] = 1  
    if dfMAST[cond_60].shape[0] > 0:
        weak_60[gids[idd]] = 1  
    if dfMAST[cond_90].shape[0] > 0:
        weak_90[gids[idd]] = 1  
    if dfMAST[condall].shape[0] > 0:
        weakall[gids[idd]] = 1  
print(np.count_nonzero(weak_30)/len(sids))
print(np.count_nonzero(weak_60)/len(sids))
print(np.count_nonzero(weak_90)/len(sids))
print(np.count_nonzero(weakall)/len(sids))

print(np.count_nonzero(weak_90)/dfMAST.groupby('id').sum().shape[0])
print(np.count_nonzero(weakall)/dfMAST.groupby('id').sum().shape[0])

0.049107142857142856
0.05357142857142857
0.05803571428571429
0.14285714285714285
0.0046982291290206
0.011564871702204554


In [46]:
illed = df[df['KETOS'] == 1]
illed_id   = illed['id'].tolist()
illed_age = illed['age'].tolist()

sids  = illed['id'].unique().tolist() 
gids  = {sids[i] : i for i in range(len(sids))}

weak_30 = [0]*len(sids)
weak_60 = [0]*len(sids)
weak_90 = [0]*len(sids)
weakall = [0]*len(sids)

for i in range(len(illed_id)):
    idd = illed_id[i]
    age = illed_age[i]
    
    if weak_30[gids[idd]] == 1:
        continue
    
    cond_30 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 30)
    cond_60 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 60)
    cond_90 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 90)
    condall = (dfMAST['id'] == idd) & (dfMAST['age'] > age)
    
    if dfMAST[cond_30].shape[0] > 0:
        weak_30[gids[idd]] = 1  
    if dfMAST[cond_60].shape[0] > 0:
        weak_60[gids[idd]] = 1  
    if dfMAST[cond_90].shape[0] > 0:
        weak_90[gids[idd]] = 1  
    if dfMAST[condall].shape[0] > 0:
        weakall[gids[idd]] = 1  
print(np.count_nonzero(weak_30)/len(sids))
print(np.count_nonzero(weak_60)/len(sids))
print(np.count_nonzero(weak_90)/len(sids))
print(np.count_nonzero(weakall)/len(sids))

print(np.count_nonzero(weak_90)/dfMAST.groupby('id').sum().shape[0])
print(np.count_nonzero(weakall)/dfMAST.groupby('id').sum().shape[0])

0.09015777610818933
0.1322314049586777
0.16679188580015025
0.3944402704733283
0.08023129743404409
0.18973617636429346


In [48]:
illed = df[df['METR'] == 1]
illed_id   = illed['id'].tolist()
illed_age = illed['age'].tolist()

sids  = illed['id'].unique().tolist() 
gids  = {sids[i] : i for i in range(len(sids))}

weak_30 = [0]*len(sids)
weak_60 = [0]*len(sids)
weak_90 = [0]*len(sids)
weakall = [0]*len(sids)

for i in range(len(illed_id)):
    idd = illed_id[i]
    age = illed_age[i]
    
    if weak_30[gids[idd]] == 1:
        continue
    
    cond_30 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 30)
    cond_60 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 60)
    cond_90 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 90)
    condall = (dfMAST['id'] == idd) & (dfMAST['age'] > age)
    
    if dfMAST[cond_30].shape[0] > 0:
        weak_30[gids[idd]] = 1  
    if dfMAST[cond_60].shape[0] > 0:
        weak_60[gids[idd]] = 1  
    if dfMAST[cond_90].shape[0] > 0:
        weak_90[gids[idd]] = 1  
    if dfMAST[condall].shape[0] > 0:
        weakall[gids[idd]] = 1  
print(np.count_nonzero(weak_30)/len(sids))
print(np.count_nonzero(weak_60)/len(sids))
print(np.count_nonzero(weak_90)/len(sids))
print(np.count_nonzero(weakall)/len(sids))

print(np.count_nonzero(weak_90)/dfMAST.groupby('id').sum().shape[0])
print(np.count_nonzero(weakall)/dfMAST.groupby('id').sum().shape[0])

0.07551164431898377
0.12232415902140673
0.16419665960950364
0.3634438955539873
0.25225876400433683
0.5583664618720636


In [49]:
illed = df[df['RP'] == 1]
illed_id   = illed['id'].tolist()
illed_age = illed['age'].tolist()

sids  = illed['id'].unique().tolist() 
gids  = {sids[i] : i for i in range(len(sids))}

weak_30 = [0]*len(sids)
weak_60 = [0]*len(sids)
weak_90 = [0]*len(sids)
weakall = [0]*len(sids)

for i in range(len(illed_id)):
    idd = illed_id[i]
    age = illed_age[i]
    
    if weak_30[gids[idd]] == 1:
        continue
    
    cond_30 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 30)
    cond_60 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 60)
    cond_90 = (dfMAST['id'] == idd) & (dfMAST['age'] > age) &  (dfMAST['age'] < age + 90)
    condall = (dfMAST['id'] == idd) & (dfMAST['age'] > age)
    
    if dfMAST[cond_30].shape[0] > 0:
        weak_30[gids[idd]] = 1  
    if dfMAST[cond_60].shape[0] > 0:
        weak_60[gids[idd]] = 1  
    if dfMAST[cond_90].shape[0] > 0:
        weak_90[gids[idd]] = 1  
    if dfMAST[condall].shape[0] > 0:
        weakall[gids[idd]] = 1  
print(np.count_nonzero(weak_30)/len(sids))
print(np.count_nonzero(weak_60)/len(sids))
print(np.count_nonzero(weak_90)/len(sids))
print(np.count_nonzero(weakall)/len(sids))

print(np.count_nonzero(weak_90)/dfMAST.groupby('id').sum().shape[0])
print(np.count_nonzero(weakall)/dfMAST.groupby('id').sum().shape[0])

0.07216494845360824
0.10446735395189004
0.13333333333333333
0.2845360824742268
0.0701120346946151
0.1496205276472714


# Склонность коров болеть в целом

In [25]:
ILLS = ['MAST', 'LAME', 'KETOS', 'METR', 'PARES', 'ILLMISC']
illed = df[(df['event'] == 'MAST') | (df['event'] == 'LAME') |(df['event'] == 'KETOS') |(df['event'] == 'METR') | (df['event'] == 'PARES') | (df['event'] == 'ILLMISC')]
illed_id = illed['id'].unique().tolist()

dfMAST = df[df['event'] == 'MAST']
mast_id = dfMAST['id'].unique().tolist()

In [29]:
sf = df[['id','event']].groupby('id').agg(list).reset_index('id')
counter = [0, 0, 0, 0, 0, 0]
for ids in illed_id:
    ids_events = set(sf[sf['id'] == ids]['event'].sum())
    ids_ills   = int(np.count_nonzero([ill in ids_events for ill in ILLS]))
    counter[ids_ills - 1] += 1

for i in range(6):
    print(counter[i]/len(illed_id))

print('--------------')
mcounter =  [0, 0, 0, 0, 0, 0]
for ids in mast_id:
    ids_events = set(sf[sf['id'] == ids]['event'].sum())
    ids_ills   = np.count_nonzero([ill in ids_events for ill in ILLS])
    mcounter[ids_ills-1] += 1
  
for i in range(6):
    print(mcounter[i]/len(illed_id))

print('--------------')
for i in range(6):
    print(mcounter[i]/counter[i])


0.529352901934623
0.21558816989103846
0.16021792305981766
0.07593951523237714
0.018123193239937736
0.000778296642205915
--------------
0.03624638647987547
0.08483433400044474
0.1027351567711808
0.06515454747609517
0.017900822770736045
0.000778296642205915
--------------
0.06847300987187566
0.3935018050541516
0.6412213740458015
0.8579795021961932
0.9877300613496932
1.0


In [88]:
illd = df[['id','MAST', 'LAME', 'KETOS', 'METR', 'PARES', 'ILLMISC']].groupby('id').sum().reset_index('id')
n = 2
total  = len(df['id'].unique().tolist())
mu = len(mast_id)/total
nu = illd[illd['LAME'] + illd['KETOS'] + illd['METR'] + illd['PARES'] + illd['ILLMISC'] >= n].shape[0]/total
p = sum(mcounter[n::]) / sum(counter[n-1::])

print(mu)
print(p * mu / nu)

0.1805900013053126
0.21032869847219227


# Анализ по группам

In [113]:
groups = []
for grp in df['group_id'].unique().tolist():
    cond = (df['event'] == 'MAST') & (df['group_id'] == grp) 
    groups.append((grp, df[cond].shape[0]))

In [118]:
groups.sort()
groups = [grp for grp in groups if ((not (grp[0] in [2, 21, 3, 34, 44, 12, 41, 1, 42, 43, 18 , 17])) and (grp[1] > 0))]
print(groups)
val = np.array([grp[1] for grp in groups])
print(val.mean(), val.std())
val.sort()
print(val)

[(4, 634), (5, 759), (6, 598), (7, 581), (8, 277), (9, 221), (10, 525), (11, 10), (13, 17), (23, 35), (45, 217), (50, 16)]
324.1666666666667 268.0714560626617
[ 10  16  17  35 217 221 277 525 581 598 634 759]
